In [36]:
import pandas as pd
import numpy as np

In [37]:
df = pd.read_csv("bank_marketing.csv")
for col in df.select_dtypes("object").columns:
    print(col)
    print("------------------------------")
    print(df[col].value_counts())

job
------------------------------
job
admin.           10422
blue-collar       9254
technician        6743
services          3969
management        2924
retired           1720
entrepreneur      1456
self-employed     1421
housemaid         1060
unemployed        1014
student            875
unknown            330
Name: count, dtype: int64
marital
------------------------------
marital
married     24928
single      11568
divorced     4612
unknown        80
Name: count, dtype: int64
education
------------------------------
education
university.degree      12168
high.school             9515
basic.9y                6045
professional.course     5243
basic.4y                4176
basic.6y                2292
unknown                 1731
illiterate                18
Name: count, dtype: int64
credit_default
------------------------------
credit_default
no         32588
unknown     8597
yes            3
Name: count, dtype: int64
mortgage
------------------------------
mortgage
yes        21576
n

In [38]:
df.head()

,client_id,age,job,marital,education,credit_default,mortgage,month,day,contact_duration,number_contacts,previous_campaign_contacts,previous_outcome,cons_price_idx,euribor_three_months,campaign_outcome
0,0,56,housemaid,married,basic.4y,no,no,may,13,261,1,0,nonexistent,93.994,4.857,no
1,1,57,services,married,high.school,unknown,no,may,19,149,1,0,nonexistent,93.994,4.857,no
2,2,37,services,married,high.school,no,yes,may,23,226,1,0,nonexistent,93.994,4.857,no
3,3,40,admin.,married,basic.6y,no,no,may,27,151,1,0,nonexistent,93.994,4.857,no
4,4,56,services,married,high.school,no,no,may,3,307,1,0,nonexistent,93.994,4.857,no


In [39]:
# Split the data into three tables
client = df[['client_id', 'age', 'job', 'marital', 'education',
             'credit_default', 'mortgage']]
campaign = df[['client_id', 'number_contacts', 'contact_duration',
               'previous_campaign_contacts', 'previous_outcome',
               'campaign_outcome', 'month', 'day']]
economics = df[['client_id', 'cons_price_idx', 'euribor_three_months']]

In [40]:
client.head()

,client_id,age,job,marital,education,credit_default,mortgage
0,0,56,housemaid,married,basic.4y,no,no
1,1,57,services,married,high.school,unknown,no
2,2,37,services,married,high.school,no,yes
3,3,40,admin.,married,basic.6y,no,no
4,4,56,services,married,high.school,no,no


In [41]:
# Editing client dataset
client.loc[:,'education'] = client['education'].str.replace(".", "_")
client.loc[:, "education"] = client['education'].replace('unknown', np.nan)
# Cleaning job column
client.loc[:, 'job'] = client['job'].str.replace(".", "_")

# Convert mortgage and credit_default to boolean
for col in ['credit_default', 'mortgage']:
    client.loc[:, col] = client[col].map({"yes": True,
                                   "no": False,
                                   "unknown": False}).astype(bool)
    client.loc[:, col] = client[col].astype(bool)

client.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   client_id       41188 non-null  int64 
 1   age             41188 non-null  int64 
 2   job             41188 non-null  object
 3   marital         41188 non-null  object
 4   education       39457 non-null  object
 5   credit_default  41188 non-null  object
 6   mortgage        41188 non-null  object
dtypes: int64(2), object(5)
memory usage: 2.2+ MB


In [42]:
campaign.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   client_id                   41188 non-null  int64 
 1   number_contacts             41188 non-null  int64 
 2   contact_duration            41188 non-null  int64 
 3   previous_campaign_contacts  41188 non-null  int64 
 4   previous_outcome            41188 non-null  object
 5   campaign_outcome            41188 non-null  object
 6   month                       41188 non-null  object
 7   day                         41188 non-null  int64 
dtypes: int64(5), object(3)
memory usage: 2.5+ MB


In [43]:
# Editing campaign dataset
campaign = campaign.copy()
# changing campaign_outcome to boolean
campaign.loc[:, 'campaign_outcome'] = campaign['campaign_outcome'].map({"yes": True,
                                                                        "no": False}).astype(bool)
# Converting previous_outcome to binary values
campaign.loc[:, 'previous_outcome'] = campaign['previous_outcome'].map({'success': 1,
                                                                     'failure': 0,
                                                                     'nonexistent': 0}).astype(bytes)

# Adding year column
campaign['year'] = '2022'
# Converting day to string
# Addiing last_contact_date coolumn
campaign['year'] = campaign['year'].astype(str)
campaign['month'] = campaign['month'].astype(str).str.zfill(2)
campaign['day'] = campaign['day'].astype(str).str.zfill(2)

campaign['last_contact_date'] = pd.to_datetime(campaign['year'] + '-' + campaign['month'] + '-' + campaign['day'], format="%Y-%b-%d")

# Cleaning and converting outcome columns to bool
for col in ["campaign_outcome",  "previous_outcome"]:
    campaign[col] = campaign[col].astype(bool)

#campaign.drop(columns=["month", "day", "year"], inplace=True)
campaign.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   client_id                   41188 non-null  int64         
 1   number_contacts             41188 non-null  int64         
 2   contact_duration            41188 non-null  int64         
 3   previous_campaign_contacts  41188 non-null  int64         
 4   previous_outcome            41188 non-null  bool          
 5   campaign_outcome            41188 non-null  bool          
 6   month                       41188 non-null  object        
 7   day                         41188 non-null  object        
 8   year                        41188 non-null  object        
 9   last_contact_date           41188 non-null  datetime64[ns]
dtypes: bool(2), datetime64[ns](1), int64(4), object(3)
memory usage: 2.6+ MB


In [44]:
# Dropping uneccessary columns
campaign.drop(columns=["month", "day", "year"], inplace=True)

In [45]:
# Saving tables to individual csv files
client.to_csv("client.csv", index=False)
campaign.to_csv("campaign.csv", index=False)
economics.to_csv("economics.csv", index=False)